In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
schooldata_complete_df = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
schooldata_complete_df

,Student ID,student_name,gender,year,school_name,reading_score,maths_score,School ID,type,size,budget
0,0,Paul Bradley,M,9,Huang High School,96,94,0,Government,2917,1910635
1,1,Victor Smith,M,12,Huang High School,90,43,0,Government,2917,1910635
2,2,Kevin Rodriguez,M,12,Huang High School,41,76,0,Government,2917,1910635
3,3,Richard Scott,M,12,Huang High School,89,86,0,Government,2917,1910635
4,4,Bonnie Ray,F,9,Huang High School,87,69,0,Government,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12,Thomas High School,51,48,14,Independent,1635,1043130
39166,39166,Dawn Bell,F,10,Thomas High School,81,89,14,Independent,1635,1043130
39167,39167,Rebecca Tanner,F,9,Thomas High School,99,99,14,Independent,1635,1043130
39168,39168,Desiree Kidd,F,10,Thomas High School,72,77,14,Independent,1635,1043130


In [2]:
#Find total number of schools
school_num = schooldata_complete_df["school_name"].nunique()

#Find total number of students
student_num = schooldata_complete_df["Student ID"].nunique()

#Find total budget
budget_df = schooldata_complete_df["budget"].unique()
sum_budget_df = budget_df.sum()

#Calculate average math score
math_total = schooldata_complete_df["maths_score"].sum()
average_math = math_total / student_num

#Calculate average reading score
reading_total = schooldata_complete_df["reading_score"].sum()
average_reading = reading_total / student_num

#Calculate math passing percentage
mathpass_df = schooldata_complete_df.loc[schooldata_complete_df["maths_score"] > 49, ["Student ID","maths_score"]]
mathpass_percent = len(mathpass_df.maths_score) / student_num * 100

#Calculate reading passing percentage
readingpass_df = schooldata_complete_df.loc[schooldata_complete_df["reading_score"] > 49, ["Student ID","reading_score"]]
readingpass_percent = len(readingpass_df.reading_score) / student_num * 100

#Calculate overall passing percentage
overallpass_df = pd.merge(mathpass_df, readingpass_df,on="Student ID")
overallpass_percent = len(overallpass_df) / student_num * 100

In [3]:
#Create new summary dataframe
lga_summary_df = pd.DataFrame(columns=['Total Schools',
                                       'Total Students',
                                       'Total Budget',
                                       'Average Maths Score',
                                       'Average Reading Score',
                                       '% Passing Maths',
                                       '% Passing Reading',
                                       '% Overall Passing'
                                      ])

#Insert data into dataframe
lga_summary_df.at[0,'Total Schools'] = school_num
lga_summary_df.at[0,'Total Students'] = student_num
lga_summary_df.at[0,'Total Budget'] = sum_budget_df
lga_summary_df.at[0,'Average Maths Score'] = average_math
lga_summary_df.at[0,'Average Reading Score'] = average_reading
lga_summary_df.at[0,'% Passing Maths'] = mathpass_percent
lga_summary_df.at[0,'% Passing Reading'] = readingpass_percent
lga_summary_df.at[0,'% Overall Passing'] = overallpass_percent

#Format the columns for Total Students and Total Budget appropriately
lga_summary_df["Total Students"] = lga_summary_df["Total Students"].map('{:,d}'.format)
lga_summary_df["Total Budget"] = lga_summary_df["Total Budget"].astype(float).map("${:,.2f}".format)
lga_summary_df

,Total Schools,Total Students,Total Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",70.338192,69.980138,86.078632,84.426857,72.808272


In [4]:
#Find the total students per school
schoolstudentcount = schooldata_complete_df["school_name"].value_counts()

#Create DataFrame from output
schoolstudent_df = pd.DataFrame(schoolstudentcount)
schoolstudent_df = schoolstudent_df.rename(columns={"school_name": "Total Students"})

#Create DataFrame for school type
schooltype = schooldata_complete_df[["school_name","type"]]
#Drop duplicates
schooltype = schooltype.drop_duplicates(subset=['school_name','type'])
#Set index to school name
schooltype = schooltype.set_index('school_name')

#Create DataFrame for total school budget
schoolbudget = schooldata_complete_df[["school_name","budget"]]
schoolbudget = schoolbudget.drop_duplicates(subset=['school_name','budget'])
schoolbudget = schoolbudget.set_index('school_name')

#Create DataFrame for average math and reading scores for each school
schoolaverage = schooldata_complete_df[["school_name","maths_score","reading_score"]]
schoolaverage = schoolaverage.groupby("school_name")[['maths_score','reading_score']].mean()

#Create DataFrame for % Passing Maths
schoolmath_percent = schooldata_complete_df.loc[schooldata_complete_df["maths_score"] > 49, ["school_name",
                                                                                             "maths_score"]]
schoolmath_percent = schoolmath_percent["school_name"].value_counts()
schoolmath_percent = schoolmath_percent / schoolstudentcount * 100
schoolmath_percent = schoolmath_percent.rename("% Passing Maths")

#Create DataFrame for % Passing Reading
schoolreading_percent = schooldata_complete_df.loc[schooldata_complete_df["reading_score"] > 49, ["school_name",
                                                                                                  "reading_score"]]
schoolreading_percent = schoolreading_percent["school_name"].value_counts()
schoolreading_percent = schoolreading_percent / schoolstudentcount * 100
schoolreading_percent = schoolreading_percent.rename("% Passing Reading")

#Create DataFrame for % Overall Passing
school_percent = schooldata_complete_df.loc[(schooldata_complete_df["maths_score"] > 49) & 
                                                (schooldata_complete_df["reading_score"] > 49), ["school_name",
                                                                                                 "maths_score",
                                                                                                 "reading_score"]]
school_percent = school_percent["school_name"].value_counts()
school_percent = school_percent / schoolstudentcount * 100
school_percent = school_percent.rename("% Overall Passing")

#Join all the DataFrames into one summary DataFrame
schoolsummary_df = schoolstudent_df.join(schooltype)
schoolsummary_df = schoolsummary_df.join(schoolbudget)
schoolsummary_df = schoolsummary_df.join(schoolaverage)
schoolsummary_df = schoolsummary_df.join(schoolmath_percent)
schoolsummary_df = schoolsummary_df.join(schoolreading_percent)
schoolsummary_df = schoolsummary_df.join(school_percent)
#Calculate Per Student Budget
schoolsummary_df["Per Student Budget"] = schoolsummary_df["budget"] / schoolsummary_df["Total Students"]
#Rename the columns
schoolsummary_df = schoolsummary_df.rename(columns={"type": "School Type",
                                                    "budget": "Total School Budget",
                                                    "maths_score": "Average Maths Score",
                                                    "reading_score": "Average Reading Score"})
#Fix number formatting for currency
schoolsummary_df["Total School Budget"] = schoolsummary_df["Total School Budget"].astype(float).map("${:,.2f}".format)
schoolsummary_df["Per Student Budget"] = schoolsummary_df["Per Student Budget"].astype(float).map("${:,.2f}".format)
#Arrange the columns
schoolsummary_df = schoolsummary_df[["School Type","Total Students","Total School Budget","Per Student Budget",
                                     "Average Maths Score","Average Reading Score","% Passing Maths","% Passing Reading",
                                     "% Overall Passing"]]
schoolsummary_df.sort_index()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Bailey High School,Government,4976,"$3,124,928.00",$628.00,72.352894,71.008842,91.639871,87.379421,80.084405
Cabrera High School,Independent,1858,"$1,081,356.00",$582.00,71.657158,71.359526,90.850377,89.074273,80.785791
Figueroa High School,Government,2949,"$1,884,411.00",$639.00,68.698542,69.077993,81.654798,82.807731,67.650051
Ford High School,Government,2739,"$1,763,916.00",$644.00,69.091274,69.572472,82.438846,82.219788,67.469880
Griffin High School,Independent,1468,"$917,500.00",$625.00,71.788147,71.245232,91.212534,88.487738,81.335150
Hernandez High School,Government,4635,"$3,022,020.00",$652.00,68.874865,69.186408,80.949299,81.877023,66.364617
Holden High School,Independent,427,"$248,087.00",$581.00,72.583138,71.660422,89.929742,88.524590,78.922717
Huang High School,Government,2917,"$1,910,635.00",$655.00,68.935207,68.910525,81.693521,81.453548,66.712376
Johnson High School,Government,4761,"$3,094,650.00",$650.00,68.843100,69.039277,82.062592,81.978576,67.191766
Pena High School,Independent,962,"$585,858.00",$609.00,72.088358,71.613306,91.683992,86.590437,79.209979


In [5]:
#Top 5 Highest-Performing Schools (by % Overall Passing)
top5_summary_df = schoolsummary_df.sort_values(by=['% Overall Passing'], ascending=False)
top5_summary_df.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Griffin High School,Independent,1468,"$917,500.00",$625.00,71.788147,71.245232,91.212534,88.487738,81.335150
Cabrera High School,Independent,1858,"$1,081,356.00",$582.00,71.657158,71.359526,90.850377,89.074273,80.785791
Bailey High School,Government,4976,"$3,124,928.00",$628.00,72.352894,71.008842,91.639871,87.379421,80.084405
Wright High School,Independent,1800,"$1,049,400.00",$583.00,72.047222,70.969444,91.777778,86.666667,79.722222
Rodriguez High School,Government,3999,"$2,547,363.00",$637.00,72.047762,70.935984,90.797699,87.396849,79.419855


In [6]:
#Bottom 5 Lowest-Performing Schools (by % Overall Passing)
bottom5_summary_df = schoolsummary_df.sort_values(by=['% Overall Passing'], ascending=True)
bottom5_summary_df.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Hernandez High School,Government,4635,"$3,022,020.00",$652.00,68.874865,69.186408,80.949299,81.877023,66.364617
Huang High School,Government,2917,"$1,910,635.00",$655.00,68.935207,68.910525,81.693521,81.453548,66.712376
Johnson High School,Government,4761,"$3,094,650.00",$650.00,68.843100,69.039277,82.062592,81.978576,67.191766
Wilson High School,Independent,2283,"$1,319,574.00",$578.00,69.170828,68.876916,82.785808,81.296540,67.455103
Ford High School,Government,2739,"$1,763,916.00",$644.00,69.091274,69.572472,82.438846,82.219788,67.469880


In [7]:
#Create Year 9 Math DataFrame
year9_math = schooldata_complete_df.loc[schooldata_complete_df["year"] == 9, ["school_name", "maths_score"]]
year9_math = year9_math.groupby('school_name').mean()
year9_math = year9_math.rename(columns={"maths_score": "Year 9"})

#Create Year 10 Math DataFrame
year10_math = schooldata_complete_df.loc[schooldata_complete_df["year"] == 10, ["school_name", "maths_score"]]
year10_math = year10_math.groupby('school_name').mean()
year10_math = year10_math.rename(columns={"maths_score": "Year 10"})

#Create Year 11 Math DataFrame
year11_math = schooldata_complete_df.loc[schooldata_complete_df["year"] == 11, ["school_name", "maths_score"]]
year11_math = year11_math.groupby('school_name').mean()
year11_math = year11_math.rename(columns={"maths_score": "Year 11"})

#Create Year 12 Math DataFrame
year12_math = schooldata_complete_df.loc[schooldata_complete_df["year"] == 12, ["school_name", "maths_score"]]
year12_math = year12_math.groupby('school_name').mean()
year12_math = year12_math.rename(columns={"maths_score": "Year 12"})

#Merge DataFrames into a Math summary table
year_math = year9_math.merge(year10_math,on='school_name').merge(year11_math,on='school_name').merge(year12_math, on='school_name')
year_math.index.name = None
year_math

,Year 9,Year 10,Year 11,Year 12
Bailey High School,72.493827,71.897498,72.374900,72.675097
Cabrera High School,72.321970,72.437768,71.008299,70.604712
Figueroa High School,68.477804,68.331586,68.811001,69.325282
Ford High School,69.021609,69.387006,69.248862,68.617811
Griffin High School,72.789731,71.093596,71.692521,71.469178
Hernandez High School,68.586831,68.867156,69.154412,68.985075
Holden High School,70.543307,75.105263,71.640777,73.409639
Huang High School,69.081754,68.533246,69.431345,68.639316
Johnson High School,69.469286,67.990220,68.637730,69.287393
Pena High School,71.996364,72.396000,72.523438,71.187845


In [8]:
#Create a Year 9 Reading DataFrame
year9_reading = schooldata_complete_df.loc[schooldata_complete_df["year"] == 9, ["school_name", "reading_score"]]
year9_reading = year9_reading.groupby('school_name').mean()
year9_reading = year9_reading.rename(columns={"reading_score": "Year 9"})

#Create a Year 10 Reading DataFrame
year10_reading = schooldata_complete_df.loc[schooldata_complete_df["year"] == 10, ["school_name", "reading_score"]]
year10_reading = year10_reading.groupby('school_name').mean()
year10_reading = year10_reading.rename(columns={"reading_score": "Year 10"})

#Create a Year 11 Reading DataFrame
year11_reading = schooldata_complete_df.loc[schooldata_complete_df["year"] == 11, ["school_name", "reading_score"]]
year11_reading = year11_reading.groupby('school_name').mean()
year11_reading = year11_reading.rename(columns={"reading_score": "Year 11"})

#Create a Year 12 Reading DataFrame
year12_reading = schooldata_complete_df.loc[schooldata_complete_df["year"] == 12, ["school_name", "reading_score"]]
year12_reading = year12_reading.groupby('school_name').mean()
year12_reading = year12_reading.rename(columns={"reading_score": "Year 12"})

#Merge DataFrames into a Reading summary table
year_reading = year9_reading.merge(year10_reading,on='school_name').merge(year11_reading,on='school_name').merge(year12_reading, on='school_name')
year_reading.index.name = None
year_reading

,Year 9,Year 10,Year 11,Year 12
Bailey High School,70.901920,70.848265,70.317346,72.195525
Cabrera High School,71.172348,71.328326,71.201245,71.856021
Figueroa High School,70.261682,67.677588,69.152327,69.082126
Ford High School,69.615846,68.988701,70.735964,68.849722
Griffin High School,72.026895,70.746305,72.385042,69.434932
Hernandez High School,68.477569,70.621842,68.418199,69.244136
Holden High School,71.598425,71.096491,73.310680,70.481928
Huang High School,68.670616,69.516297,68.740638,68.671795
Johnson High School,68.719286,69.295029,69.969115,67.992521
Pena High School,70.949091,72.324000,71.703125,71.513812


In [9]:
#Define school spending bins
spending_bins = [0, 585, 630, 645, 680]

#Define bin labels
spending_labels = ["<$585", "$585-630", "$630-645","$645-680"]

#Convert column from object to float
schoolsummary_df["Per Student Budget"] = schoolsummary_df["Per Student Budget"].str.replace('$','').astype(float)

#Slice data and place it into bins
schoolsummary_df["Spending Ranges (Per Student)"] = pd.cut(schoolsummary_df["Per Student Budget"],
                                                           spending_bins, labels=spending_labels)

#Create GroupBy object based on Spending Ranges Per Student
spending_group = schoolsummary_df.groupby("Spending Ranges (Per Student)")

#Find the average for each spending group
spending_group[["Average Maths Score", "Average Reading Score", "% Passing Maths",
                "% Passing Reading", "% Overall Passing"]].mean()

C:\Users\JoshuaTean\AppData\Local\Temp\ipykernel_8728\185943689.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  schoolsummary_df["Per Student Budget"] = schoolsummary_df["Per Student Budget"].str.replace('$','').astype(float)


,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,71.364587,70.716577,88.835926,86.390517,76.721458
$585-630,72.065868,71.031297,91.518824,87.292423,79.876293
$630-645,69.854807,69.838814,84.686139,83.763585,71.004977
$645-680,68.884391,69.045403,81.568470,81.769716,66.756253


In [10]:
#Define school size bins
school_bins = [0,1000,2000,5000]

#Define bin labels
school_labels = ["Small (<1000)","Medium (1000-2000)","Large (2000-5000)"]

#Slice data and place it into bins
schoolsummary_df["School Size"] = pd.cut(schoolsummary_df["Total Students"], school_bins, labels=school_labels)

#Create GroupBy object based on School Size
school_group = schoolsummary_df.groupby("School Size")

#Find the average for each school size
school_group[["Average Maths Score", "Average Reading Score", "% Passing Maths",
              "% Passing Reading", "% Overall Passing"]].mean()

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),72.335748,71.636864,90.806867,87.557513,79.066348
Medium (1000-2000),71.421650,70.720164,89.846560,86.714149,78.039785
Large (2000-5000),69.751809,69.576052,84.252804,83.301185,70.293507


In [11]:
#Create GroupBy object based on school type
school_type = schoolsummary_df.groupby("School Type")

#Find the average for each school type
school_type[["Average Maths Score", "Average Reading Score", "% Passing Maths",
             "% Passing Reading", "% Overall Passing"]].mean()

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Type,,,,,
Government,69.834806,69.675929,84.462375,83.587562,70.698993
Independent,71.368822,70.718933,89.204043,86.247789,76.973340
